In [1]:
import numpy as np
from load import load_data
from convert import convert
from pmc2 import *
from brute_force import planes_def

In [2]:
#%matplotlib widget
#%matplotlib inline
%matplotlib qt
#%matplotlib gtk
import matplotlib.pyplot as plt
import matplotlib.font_manager
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
plt.rcParams.update({
    'pgf.rcfonts': False,
})
plt.rcParams['font.family'] = 'serif'
#from matplotlib.backends.backend_pgf import FigureCanvasPgf
#matplotlib.backend_bases.register_backend('pdf', FigureCanvasPgf)

In [3]:
alf_inter = 1
alf_plane = 0.5

# Input

In [47]:
data = load_data(13)
d = convert(data)
P1, P2, Rsqu, Rsqu_P1, Rsqu_P2 = planes_def(data,d)
print(Rsqu, Rsqu_P1, Rsqu_P2)


[ 52.46933814  37.2983483   -1.0470212  112.50663198  12.49566163
  10.10514832]
[16.88597192 11.6163253  -1.10117029 12.66518231 33.05403686 32.77526363]
[0.96633366 1.0439162  1.0841561  1.10958772 1.03551236 1.09345782
 1.04964286 1.02067013 1.08344438 0.97363598 1.03801983 0.93304022
 0.92821143 0.97649932 1.01492671 0.96490702 0.95240209 1.09868637
 0.92636193 0.99284695 0.98961017 0.81925684 0.97718586 0.87849996
 1.01668786 1.04631928 1.06704747 1.18121699 0.90257753 0.98047605
 1.11874289 1.23389947 1.08075617 0.72555758 1.2030343  1.13559153
 0.83209907 1.03169349 0.9979721  1.09739663 0.96738467 1.80151111
 1.044395  ]
4.045454254923152
0.9579308615904037 -0.05881087490091552 -0.06118837085400375


In [48]:
eq_pts_transisiton_P12 = np.zeros((6,3,3))
for i in range(6):
    eq_pts_transisiton_P12[i] = np.array([get_plane_normal_6_cycle(P1,i),get_plane_normal_6_cycle(P1,i-1),get_plane_normal_6_cycle(P2,i)])

ones = np.ones((6,3))
pts_transisiton_P12 = np.linalg.solve(eq_pts_transisiton_P12, ones).transpose()

In [49]:
eq_pts_P2 = np.array([get_plane_normal_6_cycle(P2,0),get_plane_normal_6_cycle(P2,1),get_plane_normal_6_cycle(P2,2)])
pts_P2 = np.linalg.solve(eq_pts_P2,np.ones((3,1)))

In [50]:
p_trans = (pts_transisiton_P12[0,:]+pts_transisiton_P12[1,:]+pts_transisiton_P12[2,:])/3
#print(p_trans)

In [51]:
sig_pts = np.concatenate((P1.pts,P2.pts),axis=1) #miss pts_P2
pts_with_largest_coord = sig_pts[:,np.argmax(np.max(sig_pts, axis=0))]
dist = np.dot(np.ones(3)/np.sqrt(3),pts_with_largest_coord)*1.1
#print(dist)

In [52]:
pts_P1_far = p_plane_intersection_6(P1,dist)

In [53]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')
ax.plot(*P1.pts,'k.')
ax.plot(*P2.pts,'k.')
#ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')
ax.plot(*pts_P2,'ko',alpha=0)
ax.plot(*pts_transisiton_P12,'ro',alpha=alf_inter)
ax.plot(*pts_P1_far,'bo',alpha=alf_inter)
def get_verts(coord):
    return [list(zip(*cord))]

## Draw P2 plane

In [54]:
for i in range(6):
    cord = np.zeros((3,3))
    cord[:,0] = pts_P2[:,0]
    cord[:,1] = pts_transisiton_P12[:,i]
    cord[:,2] = pts_transisiton_P12[:,(i+1)%6]
    
    verts = get_verts(cord)
    poly = Poly3DCollection(verts,linewidths=1, alpha=alf_plane)
    poly.set_facecolor('r')
    ax.add_collection3d(poly)

## Draw P1 plane

In [55]:
for i in range(6):
    cord = np.zeros((3,4))
    cord[:,0] = pts_transisiton_P12[:,i]
    cord[:,1] = pts_transisiton_P12[:,(i+1)%6]
    cord[:,2] = pts_P1_far[:,(i+1)%6]
    cord[:,3] = pts_P1_far[:,i%6]
    
    verts = get_verts(cord)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.5)
    poly.set_facecolor('b')
    ax.add_collection3d(poly)

In [56]:
fig.show()
#fig.savefig('3D.pdf')

# 2D view

In [14]:
new_base = np.zeros((3,3))

new_z = np.array([1,1,1])
new_y = np.array([-1,-1,2])
new_x = np.array([-1,1,0])
new_base[:,0] = new_x /np.linalg.norm(new_x )
new_base[:,1] = new_y /np.linalg.norm(new_y )
new_base[:,2] = new_z /np.linalg.norm(new_z )
new_base_transform = np.linalg.inv(new_base)

In [15]:
#new_sig = new_base_transform @ sig_pts
new_transi = new_base_transform @ pts_transisiton_P12
new_P2 = new_base_transform @ pts_P2
new_far_P1 = new_base_transform @ pts_P1_far

In [16]:
transi_dist_min = np.min(new_transi[2,:])
transi_dist_max = np.max(new_transi[2,:])
transi_dist_mid = (transi_dist_min+transi_dist_max)/2
near_inter = p_plane_intersection_6(P2,transi_dist_min*0.4)
near_transP2 = p_plane_intersection_6(P2,transi_dist_mid*1)
near_transP1 = p_plane_intersection_6(P1,transi_dist_mid*1)
new_near = new_base_transform @ near_inter
new_transP2 = new_base_transform @ near_transP2
new_transP1 = new_base_transform @ near_transP1

In [17]:
transi_dist_max = np.max(new_transi[2,:])
t = 0.5
dist = transi_dist_max*t + transi_dist_min*(1-t)
#offset = 0 if np.allclose(new_transi[0,0],transi_dist_max) else 2
offset = 0 if p_trans[0]>p_trans[1] else 4
middle_inter = p_plane_intersection_12(P1,P2,dist,offset)
new_middle = new_base_transform @ middle_inter

In [18]:
o_sig = d['o'][:,:3].transpose()
o_pts = np.zeros(o_sig.shape)
o_pts[0,:] = o_sig[1,:]
o_pts[1,:] = o_sig[2,:]
o_pts[2,:] = o_sig[0,:]
new_o_pts = new_base_transform @ o_pts
print(o_pts)

[[ 58.1  67.7  90.9 137.8 205.4 281.7 322.1  82.7 132.9 185.6 212.1 255.8
  307.  107.5 146.5 210.  278.9 319.5 365.2 126.  170.4 257.3 356.  385.9]
 [ 45.   45.   45.   45.   45.   45.   45.   60.   60.   60.   60.   60.
   60.   75.   75.   75.   75.   75.   75.  100.  100.  100.  100.  100. ]
 [312.7 325.5 338.9 349.1 359.1 368.2 354.  353.4 385.2 401.8 403.7 402.4
  382.3 404.3 416.7 437.6 439.6 429.7 452.2 461.  492.8 496.  521.4 511.7]]


In [19]:
#p = np.unique(d['o'][:,3])
#pts_tt1p = p_planes(p[0],P1,P2,p_trans)
#pts_tt2p = p_planes(p[1],P1,P2,p_trans)
#pts_tt3p = p_planes(p[3],P1,P2,p_trans)
#pts_tt1 = p_planes(p[2],P1,P2,p_trans)

In [20]:
#new_tt1p = new_base_transform @ pts_tt1p
#new_tt2p = new_base_transform @ pts_tt2p
#new_tt3p = new_base_transform @ pts_tt3p
#new_tt1 = new_base_transform @ pts_tt1

In [21]:
plt.close()
#plt.axis('equal')
#plt.plot(new_o_pts[0,:3],new_o_pts[1,:3],'r*')
#plt.plot(new_o_pts[0,3],new_o_pts[1,3],'b.')
#plt.plot(new_o_pts[0,4:6],new_o_pts[1,4:6],'gp')
#plt.plot(new_o_pts[0,6],new_o_pts[1,6],'yX')
#plt.fill(new_P12[0,:],new_P12[1,:],edgecolor='k',fill=False)
plt.fill(new_far_P1[0,:],new_far_P1[1,:],edgecolor='k',fill=False)
plt.fill(new_near[0,:],new_near[1,:],edgecolor='k',fill=False)
plt.fill(new_transP2[0,:],new_transP2[1,:],edgecolor='r',fill=False)
plt.fill(new_transP1[0,:],new_transP1[1,:],edgecolor='b',fill=False)
plt.fill(new_middle[0,:],new_middle[1,:],edgecolor='k',fill=False)
#plt.fill(new_tt1p[0,:],new_tt1p[1,:],edgecolor='r',fill=False)
#plt.fill(new_tt2p[0,:],new_tt2p[1,:],edgecolor='b',fill=False)
#plt.fill(new_tt3p[0,:],new_tt3p[1,:],edgecolor='g',fill=False)
#plt.fill(new_tt1[0,:],new_tt1[1,:],edgecolor='y',fill=False)

In [22]:
x = np.linspace(0,80)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')

x = np.linspace(-110,0)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')

In [23]:
plt.axis('equal')
plt.axis('off')
plt.show()

In [14]:
plt.close()